In [2]:
#importing all the dependencies
import pandas as pd
import numpy as np
import os
import zipfile
import sqlalchemy as sal
from sqlalchemy import create_engine,text
from dotenv import load_dotenv

In [ ]:
#extracting the downloaded dataset 
zip_path=r'C:\Users\TE681SG\OneDrive - EY\Desktop\Ad Hoc\Harshal Portfolio Projects\GIT\ETL\data\orders.csv.zip'
extract_folder=r'C:\Users\TE681SG\OneDrive - EY\Desktop\Ad Hoc\Harshal Portfolio Projects\GIT\ETL\data'

with zipfile.ZipFile(zip_path,'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print('Zip file extracted successfully !')

In [ ]:
#reading the CSV and handling null balues
df=pd.read_csv(r'C:\Users\TE681SG\OneDrive - EY\Desktop\Ad Hoc\Harshal Portfolio Projects\GIT\ETL\data\orders.csv')
df['Ship Mode'].unique()

# output -> <StringArray>
# [  'Second Class', 'Standard Class',  'Not Available',        'unknown',
#     'First Class',              nan,       'Same Day']


#here i want to handle the values like Not Available , unknown etc
df=pd.read_csv(r'C:\Users\TE681SG\OneDrive - EY\Desktop\Ad Hoc\Harshal Portfolio Projects\GIT\ETL\data\orders.csv',na_values=['Not Available','unknown'])
#df['Ship Mode'].unique()

# output -> <StringArray>
# ['Second Class', 'Standard Class', nan, 'First Class', 'Same Day']
# Length: 5, dtype: str

In [ ]:
#renaming the column name , making them standardized
#df.rename(columns={'Order Id':'order_id','City':'city'}) --not a good way to rename
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')

In [ ]:
#derive new columns discount , sale price and profit
df['discount']=df['list_price']*df['discount_percent']*.01
df['sale_price']=df['list_price'] - df['discount']
df['profit']=df['sale_price'] - df['cost_price'] 
df

In [ ]:
#convert order date from object / str to datetime
df['order_date']=pd.to_datetime(df['order_date'],format='%Y-%m-%d')

In [ ]:
#drop cost price , list price and discount percent columns
df.drop(columns=['list_price','discount_percent','cost_price'],inplace=True)
df.head(5)

In [3]:
#encoding the credentials into .env and calling them here
load_dotenv()
server=os.getenv('DB_SERVER')
database=os.getenv('DB_DATABASE')
driver=os.getenv('DB_DRIVER')

# URL-encode driver name and use Trusted_Connection=yes
conn_str = f"mssql+pyodbc://@{server}/{database}?driver={driver.replace(' ', '+')}&Trusted_Connection=yes"

# Create engine
engine = create_engine(conn_str)

# Test connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print("Connection successful:", result.fetchone())


Connection successful: (1,)


C:\Users\TE681SG\AppData\Local\Temp\ipykernel_9968\1578865741.py:14: SAWarning: Unrecognized server version info '17.0.1050.2'.  Some SQL Server features may not function properly.
  with engine.connect() as conn:


In [ ]:
#load data into sql server
# CREATE TABLE df_orders (
#     [order_id] INT PRIMARY KEY,
#     [order_date] DATE,
#     [ship_mode] VARCHAR(20),
#     [segment] VARCHAR(20),
#     [country] VARCHAR(20),
#     [city] VARCHAR(20),
#     [state] VARCHAR(20),
#     [postal_code] VARCHAR(20),
#     [region] VARCHAR(20),
#     [category] VARCHAR(20),
#     [sub_category] VARCHAR(20),
#     [product_id] VARCHAR(50),
#     [quantity] INT,
#     [discount] DECIMAL(7,2),
#     [sale_price] DECIMAL(7,2),
#     [profit] DECIMAL(7,2)
# );  
# --- we are using this DDL because if we directly use if_exists='replace' in the below code , 
#     it gives the datatype as max i.e varchar which we do not need , and we need proper datatype
#     so we create the table DDL and then just append the data in it.

df.to_sql('df_orders',con=conn_str,index=False,if_exists='append')


NEXT STAGE :
ANALYSIS AND FINDING BUSINESS ANSWERS WILL BE DONE IN SQL , SEE SQL_ANALYSIS FILE FOR DETAILS. 